# CP 321 – Data Visualization

## Project

In [ ]:
# Imports
import pandas as pd
import numpy as np
from dash import dcc, html
from dash import Input, Output, ctx
import plotly.express as px
from random import randint
import dash

In [ ]:
JS_SCRIPTS = [
    "https://cdn.jsdelivr.net/npm/@tailwindcss/browser@4",
    "assets/app.js",
]
SYTLE_SHEETS = [
    "https://cdn.jsdelivr.net/npm/daisyui@5",
    "https://cdn.jsdelivr.net/npm/daisyui@5/themes.css",
    "https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0/css/all.min.css",
]

## Model Class 

In [ ]:
pd.options.mode.chained_assignment = None
REDUNDANT_COLS = [
    "DGUID",
    "Age (2)",
    "UOM",
    "UOM_ID",
    "SCALAR_FACTOR",
    "SCALAR_ID",
    "STATUS",
    "SYMBOL",
    "TERMINATED",
    "DECIMALS",
    "VECTOR",
    "REF_DATE",
    "COORDINATE",
]


class Model:
    def __init__(self, url):
        self.url = url
        self.data = self._load_data()
        self._preprocess_data()

    def _load_data(self):
        return pd.read_csv(self.url)

    def _preprocess_data(self):
        self.data = self.data.drop(columns=REDUNDANT_COLS)
        self.data.columns = [
            "geo",
            "education",
            "major",
            "occupation",
            "gender",
            "value",
        ]

        self.data["gender"] = self.data.gender.apply(self._clean_gender_col)
        self.data["noc"] = self.data.occupation.apply(self._clean_noc_codes)
        self.data["heir"] = self.data.noc.apply(self._clean_hierarchy_num)
        self.data["occupation_c"] = self.data.occupation.apply(self._clean_occupations)
        self.data["education_up"] = self.data.education.apply(self._update_education)

    def _clean_gender_col(self, value):
        return value[:-1]

    def _clean_noc_codes(self, value):
        noc = value.split(" ")[0]
        if not noc.isdigit():
            return np.nan
        return noc

    def _clean_hierarchy_num(self, value):
        if type(value) is float:
            return 0
        return len(value)

    def _clean_occupations(self, value):
        value = value.split(" ")
        value.pop(0)
        return " ".join(value)

    def get_province_list(self):
        return sorted(self.data.geo.unique())

    def get_field_list(self):
        return sorted(self.data[self.data.heir == 1].occupation_c.unique())

    def get_gender_distribution_by_field(self, field, province_list):
        temp_df = (
            self.data[self.data.heir == 1]
            .groupby(["geo", "occupation_c", "gender"], as_index=False)
            .value.sum()
        )

        temp_df = temp_df[temp_df.occupation_c == field]
        return temp_df[temp_df.geo.isin(province_list)]

    def get_gender_distribution_arsc(self, field):
        temp_df = (
            self.data[self.data.heir == 1]
            .groupby(["occupation_c", "gender"], as_index=False)
            .value.sum()
        )
        return temp_df[temp_df.occupation_c == field]

    def _get_list_of_all_level_occupation(self, level=5):
        return self.data[self.data.heir == level].occupation_c.unique()

    def _get_list_of_engineering(self):
        ENGINEERING_OCCUPATIONS = {
            "electrical and electronics engineers",
            "mechanical engineers",
            "computer engineers",
        }
        occupations = self._get_list_of_all_level_occupation(5)
        essentail_cols = []
        for occupation in occupations:
            lower_occupation = occupation.lower()
            if any(eng in lower_occupation for eng in ENGINEERING_OCCUPATIONS):
                essentail_cols.append(occupation)
        return essentail_cols

    def _update_education(self, value):
        if value == "Postsecondary certificate, diploma or degree":
            return "With Postsecondary Education"
        return "Without Postsecondary Education"

    def get_engineering_df(self):
        eng_df = self.data[self.data.occupation_c.isin(self._get_list_of_engineering())]
        eng_df["education_filtered"] = eng_df["education"].apply(self._update_education)
        return eng_df

    def get_list_of_essential_occ(self):
        ESSENTIAL_OCCUPATIONS = {"nurse", "police", "fire"}
        essentail_cols = []
        occupations = self._get_list_of_all_level_occupation(5)
        for occupation in occupations:
            lower_occupation = occupation.lower()
            if (
                any(
                    essential in lower_occupation for essential in ESSENTIAL_OCCUPATIONS
                )
                and "nursery" not in lower_occupation
            ):
                essentail_cols.append(occupation)
        return essentail_cols

    def get_essentails_df(self):
        essential_df = self.data[
            self.data.occupation_c.isin(self.get_list_of_essential_occ())
        ]
        return essential_df

    def get_essentails_df_whole(self):
        df = self.get_essentails_df()
        df = df.groupby(["geo", "occupation_c"], as_index=False).value.sum()
        return df

    def get_essential_by_profession(self, profession):
        df = df = self.get_essentails_df()
        df = df[df.occupation_c == profession]
        return df

    def filter_df_by_region_and_edu(self, region, education=None):
        df = self.data[self.data.heir == 1]
        if not education:
            return df[df.geo == region]
        return df[(df.geo == region) & (df.education_up == education)]

    def total_employes(self, df):
        return f"{df.value.sum():,d}"

    @staticmethod
    def highest_and_lowest_field(df: pd.DataFrame):
        temp_df = df.groupby(["occupation_c"], as_index=False).value.sum()
        temp_df = temp_df.sort_values("value", ascending=False)
        highest = temp_df.iloc[0]
        return (
            highest["occupation_c"],
            highest["value"],
        )

    @staticmethod
    def gender_analysis(df: pd.DataFrame):
        temp_df = df.groupby(["occupation_c", "gender"], as_index=False).value.sum()
        temp_df = temp_df.pivot(index="occupation_c", columns="gender", values="value")
        temp_df["ratio"] = (temp_df.Men / temp_df.Women) * 100
        return temp_df

    @staticmethod
    def feild_highest_val(df: pd.DataFrame, col="Men", ascen=False):
        df = df.sort_values(by=col, ascending=ascen)
        highest = df.iloc[0]
        return highest.name, int(highest[col])

## Graphing Functions

In [ ]:
COLOR_PALLET = px.colors.qualitative.Safe


def add_tilte(fig, title):
    fig.update_layout(
        title=dict(
            text=title,
            font=dict(size=15, family="Arial", color="#2a4978"),
        )
    )
    return fig


def add_horizontal_legend(fig):
    fig.update_layout(
        legend=dict(
            orientation="h",
            title=None,
            y=1,
            x=1,
            xanchor="right",
            yanchor="bottom",
        ),
        legend_bordercolor="black",
        legend_borderwidth=1,
    )
    return fig


def create_gender_distribution_plot(df, field):
    fig = px.bar(
        df,
        x="geo",
        y="value",
        color="gender",
        barmode="group",
        log_y=True,
        template="plotly_white",
        color_discrete_sequence=COLOR_PALLET,
    )
    fig.update_layout(xaxis={"categoryorder": "total descending"})
    fig.update_layout(
        xaxis_tickangle=-30,
        xaxis=dict(
            categoryorder="total descending",
            tickangle=-30,
            title="<b>Administrative Units</b>",
            tickfont=dict(size=12),
            title_font=dict(size=12),
        ),
        yaxis=dict(
            title="<b>Number of People (Log Scale)</b>",
            tickfont=dict(size=12),
            title_font=dict(size=12),
            showgrid=True,
            gridcolor="rgba(200, 200, 200, 0.3)",
        ),
        height=600,
        margin=dict(l=0, r=25, t=0, b=0),
    )
    fig.update_traces(
        textfont_size=10,
        textangle=0,
        textposition="outside",
        cliponaxis=False,
        texttemplate="%{y:.2s}",
        hovertemplate="<b>Region:</b> %{x}<br>" "<b>Count:</b> %{y}<extra></extra>",
    )

    fig = add_horizontal_legend(fig)
    # fig = add_tilte(fig, f"<b>Gender Composition of  {field} Across Units</b>")
    return fig


def create_gender_distribution_pie(df, field):
    fig = px.pie(
        df,
        values="value",
        names="gender",
        color_discrete_sequence=COLOR_PALLET,
        hole=0.5,
        width=350,
    )

    # fig.update_layout(legend=dict(orientation="v"))
    fig = add_horizontal_legend(fig)
    fig.update_traces(
        textposition="inside", textinfo="percent+label", insidetextfont=dict(size=12)
    )

    fig.update_layout(
        uniformtext_minsize=12,
        uniformtext_mode="hide",
        margin=dict(l=10, r=10, t=10, b=10),
    )

    return fig


def create_tree_map(df):
    midpoint = np.median(df["value"])

    fig = px.treemap(
        df,
        path=[px.Constant("Canada"), "geo", "occupation_c", "education_filtered"],
        values="value",
        color="occupation_c",
        color_discrete_sequence=COLOR_PALLET,
        color_continuous_midpoint=midpoint,
        height=750,
        template="plotly_white",
    )

    fig.update_layout(
        margin=dict(t=0, l=0, r=0, b=0),
        hoverlabel=dict(bgcolor="white", font_size=14, font_family="Helvetica"),
        uniformtext=dict(minsize=10, mode="hide"),
    )

    fig.update_traces(
        root_color="lightgrey",
        textinfo="label+value+percent parent",
        texttemplate="<b>%{label}</b><br>(%{percentParent:.1%})",
        textposition="middle left",
        marker=dict(line=dict(width=0.5, color="black")),
        hovertemplate="<b>%{label}</b><br>Count: %{value:,}<br>%{percentParent:.1%} of parent<extra></extra>",
        textfont_size=15,
    )

    return fig


def create_polar_essentails(df):
    fig = px.line_polar(
        df,
        r="value",
        theta="occupation_c",
        color="geo",
        color_discrete_sequence=COLOR_PALLET,
        template="plotly_white",
        labels={"value": "Count (log)", "occupation_c": "Occupation Category"},
        hover_name="geo",
        hover_data={"value": ":.2f"},
        log_r=True,
        start_angle=30,
        line_close=True,
        line_shape="spline",
    )

    fig.update_layout(
        title=dict(
            font=dict(size=22, family="Arial", color="#2a4978"),
            x=0.5,
            xanchor="center",
        ),
        font=dict(family="Arial", size=13, color="#333"),
        polar=dict(
            radialaxis=dict(
                angle=45,
                showline=True,
                linewidth=2,
                gridcolor="rgba(10,10,10,0.2)",
                tickangle=0,
            ),
            angularaxis=dict(
                linewidth=2,
                gridcolor="rgba(10,10,10,0.2)",
                tickfont=dict(size=12),
            ),
        ),
        legend=dict(
            orientation="h",
            title=None,
            y=-0.45,
            x=0.5,
            xanchor="center",
            yanchor="bottom",
            bordercolor="black",
            borderwidth=1,
            font=dict(size=12),
        ),
        margin=dict(l=0, r=0, t=20, b=50),
    )

    return fig


def create_essentials_pie(df):
    fig = px.pie(
        df,
        values="value",
        names="geo",
        color_discrete_sequence=COLOR_PALLET,
        hole=0.0,
        labels={"value": "Count"},
        template="plotly_white",
    )

    fig.update_layout(
        font_family="Helvetica Neue",
        margin=dict(t=10, b=10, l=10, r=10),
        showlegend=True,
        legend_title_text="Regions",
        legend_font_size=12,
        legend_title_font_size=14,
    )

    # fig = add_horizontal_legend(fig)

    fig.update_traces(
        textposition="inside",
        textinfo="percent+label",
        insidetextfont=dict(color="black", size=14, family="Arial"),
        marker=dict(line=dict(color="white", width=2)),
        hovertemplate="<b>%{label}</b><br>Count: %{value}<br>Percentage: %{percent}<extra></extra>",
        pull=[0.1 if i == df["value"].idxmax() else 0.02 for i in range(len(df))],
    )

    fig.update_layout(
        legend=dict(
            orientation="v",
        ),
        legend_bordercolor="black",
        legend_borderwidth=1,
        margin=dict(l=10, r=0, t=20, b=50),
    )

    return fig


def create_pie_chart(df, title: str = "Occupation Distribution"):

    # Create figure
    fig = px.pie(
        df,
        names="occupation_c",
        values="value",
        hole=0.75,
        color_discrete_sequence=COLOR_PALLET,
    )

    # Configure layout
    fig.update_layout(
        margin=dict(t=10, b=10, l=10, r=10),
        showlegend=True,
        legend=dict(
            font=dict(size=10),
            orientation="v",
            xanchor="left",
        ),
        legend_bordercolor="black",
        legend_borderwidth=1,
        hoverlabel=dict(bgcolor="white", font_size=12, font_family="Arial"),
    )

    # Configure traces
    text_info = "percent+label"
    fig.update_traces(
        textposition="none",
        textinfo=text_info,
        hovertemplate="<b>%{label}</b><br>Value: %{value:,}<br>Percentage: %{percent}<extra></extra>",
        textfont=dict(size=12),
        marker=dict(line=dict(color="#FFFFFF", width=1)),
    )

    return fig

## Components

In [ ]:
def render_dashboard_header():
    return html.Div(
        className="container mx-auto mb-10 mt-5",
        children=[
            html.H1(
                className="text-2xl font-bold text-gray-800",
                children="Workforce Distribution and Occupational Trends Across Canadian Provinces and Territories",
            ),
            html.Span(
                className="text-gray-600",
                children=[
                    "Data Source: Statistics Canada, 2021 Census (Table: 98-10-0404-01)",
                ],
            ),
        ],
    )

## View 1

In [ ]:
STAT_CSS = "card shadow-md hover:shadow-xl rounded-2xl p-6  transition duration-300 border border-gray-100"
STAT_LABEL = "text-sm font-bold text-primary mb-2"
STAT_VAL = "text-2xl font-bold text-gray-800 mt-1"
STAT_COUNT = "text-sm mt-2 text-success"


class GeneralAnalysis:
    def __init__(self, model: Model, app):
        self.model = model
        self.app = app
        self.layout = self._create_layout()
        print(id(self.model))
        self._register_callbacks()

    def _create_layout(self):
        return html.Div(
            className="container mx-auto mb-10",
            children=[self._render_filter_section(), self._render_stats()],
        )

    def _render_filter_section(self):
        return html.Div(
            className="grid grid-cols-2 mx-auto card shadow-md hover:shadow-xl rounded-2xl p-6 border border-gray-100 mb-4",
            children=[
                self._render_region_picker(),
                self._render_education_filter(),
            ],
        )

    def _render_stats(self):
        return html.Div(
            className="grid lg:grid-cols-4 gap-5",
            children=[
                self._render_total_emp_stat(),
                self._render_highest_emp_field(),
                self._render_field_highest_male(),
                self._render_field_highest_female(),
                self._render_field_employment(),
                self._render_field_highest_sex_ratio(),
                self._render_field_lowest_sex_ratio(),
            ],
        )

    def _render_total_emp_stat(self):
        return html.Div(
            className=STAT_CSS,
            children=[
                html.Label(
                    className=STAT_LABEL,
                    children="Total Employed Population",
                ),
                html.H3(
                    className=STAT_VAL,
                    id="total-emp-stat",
                    children="234,424",
                ),
            ],
        )

    def _render_highest_emp_field(self):
        return html.Div(
            className=STAT_CSS,
            children=[
                html.Label(
                    className=STAT_LABEL,
                    children="Field With Highest Employees",
                ),
                html.H3(
                    className=STAT_VAL,
                    id="highest-employe-field",
                    children="Legislative and senior management occupations",
                ),
                html.Label(
                    id="highest-employe-count",
                    className=STAT_COUNT,
                    children="43,1234",
                ),
            ],
        )

    def _render_field_highest_male(self):
        return html.Div(
            className=STAT_CSS,
            children=[
                html.Label(
                    className=STAT_LABEL,
                    children="Field With Highest Male Employees",
                ),
                html.H3(
                    className=STAT_VAL,
                    id="highest-male-field",
                    children="Legislative and se",
                ),
                html.Label(
                    id="highest-male-count",
                    className=STAT_COUNT,
                    children="43,1234",
                ),
            ],
        )

    def _render_field_highest_female(self):
        return html.Div(
            className=STAT_CSS,
            children=[
                html.Label(
                    className=STAT_LABEL,
                    children="Field With Highest Female Employees",
                ),
                html.H3(
                    className=STAT_VAL,
                    id="highest-female-field",
                    children="Legislative and se",
                ),
                html.Label(
                    id="highest-female-count",
                    className=STAT_COUNT,
                    children="43,1234",
                ),
            ],
        )

    def _render_field_highest_sex_ratio(self):
        return html.Div(
            className=STAT_CSS,
            children=[
                html.Label(
                    className=STAT_LABEL,
                    children="Field With Highest Sex Ratio",
                ),
                html.H3(
                    className=STAT_VAL,
                    id="highest-sex-ratio-field",
                    children="Legislative and se",
                ),
                html.Label(
                    id="highest-sex-ratio-count",
                    className=STAT_COUNT + " hidden",
                    children="43,1234",
                ),
            ],
        )

    def _render_field_lowest_sex_ratio(self):
        return html.Div(
            className=STAT_CSS,
            children=[
                html.Label(
                    className=STAT_LABEL,
                    children="Field With Lowest Sex Ratio",
                ),
                html.H3(
                    className=STAT_VAL,
                    id="lowest-sex-ratio-field",
                    children="Legislative and se",
                ),
                html.Label(
                    id="lowest-sex-ratio-count",
                    className=STAT_COUNT + " hidden",
                    children="43,1234",
                ),
            ],
        )

    def _render_field_employment(self):
        return html.Div(
            className=STAT_CSS + " col-span-3 row-span-2",
            children=[
                html.Label(
                    className=STAT_LABEL,
                    children="Regional Employment Distribution by Field",
                ),
                dcc.Graph(id="graph-employment-by-field"),
            ],
        )

    def _render_region_picker(self):
        province_list = self.model.get_province_list()
        return html.Div(
            className="z-[2] ",
            children=[
                html.Label(
                    className="label-text font-medium text-gray-700 mb-2 font-bold",
                    children="Region",
                ),
                dcc.Dropdown(
                    id="province-selector-an1",
                    value=province_list[8],
                    options=province_list,
                    clearable=False,
                    className="w-96",
                ),
            ],
        )

    def _render_education_filter(self):
        return html.Div(
            className="",
            children=[
                html.Label(
                    className="label-text font-medium text-gray-700 mb-2 font-bold",
                    children="Education Level",
                ),
                dcc.RadioItems(
                    id="radio-education",
                    options={
                        "All": "All",
                        "With Postsecondary Education": "With Postsecondary Education",
                        "Without Postsecondary Education": "Without Postsecondary Education",
                    },
                    value="All",
                ),
            ],
        )

    def _register_callbacks(self):

        @self.app.callback(
            [
                Output("total-emp-stat", "children"),
                Output("highest-employe-field", "children"),
                Output("highest-employe-count", "children"),
                Output("highest-male-field", "children"),
                Output("highest-male-count", "children"),
                Output("highest-female-field", "children"),
                Output("highest-female-count", "children"),
                Output("highest-sex-ratio-field", "children"),
                Output("highest-sex-ratio-count", "children"),
                Output("lowest-sex-ratio-field", "children"),
                Output("lowest-sex-ratio-count", "children"),
                Output("graph-employment-by-field", "figure"),
            ],
            [
                Input("province-selector-an1", "value"),
                Input("radio-education", "value"),
            ],
        )
        def handle_region_change(region, education):
            if education == "All":
                df = self.model.filter_df_by_region_and_edu(region)
            else:
                df = self.model.filter_df_by_region_and_edu(region, education)

            total_emps = self.model.total_employes(df)

            (
                hf,
                hc,
            ) = Model.highest_and_lowest_field(df)

            fig = create_pie_chart(df)
            gender_data = Model.gender_analysis(df)
            h_male_f, h_male_c = Model.feild_highest_val(gender_data, "Men")
            h_female_f, h_female_c = Model.feild_highest_val(gender_data, "Women")
            h_ratio_f, h_ratio_v = Model.feild_highest_val(gender_data, "ratio", True)
            l_ratio_f, l_ratio_v = Model.feild_highest_val(gender_data, "ratio", False)
            return (
                total_emps,
                hf,
                f"{hc:,d}",
                h_male_f,
                f"{h_male_c:,d}",
                h_female_f,
                f"{h_female_c:,d}",
                h_ratio_f,
                h_ratio_v,
                l_ratio_f,
                l_ratio_v,
                fig,
            )

## Veiw 2

In [ ]:
class EssentialSevericeDistribution:
    def __init__(self, model, app):
        self.model = model
        self.app = app
        self.layout = self._create_layout()
        self._register_callbacks()
        print(id(self.model))

    def _create_layout(self):
        return html.Div(
            className="container mx-auto card shadow-md hover:shadow-xl rounded-2xl p-6  transition duration-300 border border-gray-100 my-20",
            children=[
                html.Div(
                    children=[
                        html.H2(
                            className="text-xl font-bold mb-5 text-primary",
                            children="Distribution of Essential Service Human Resources Across Administrative Units",
                        ),
                        html.Div(
                            className="grid grid-cols-1 lg:grid-cols-3 gap-5",
                            children=[
                                self._render_buttons(),
                                self._render_distribution_chart(),
                            ],
                        ),
                        self._render_reset_button(),
                    ]
                )
            ],
        )

    def _render_reset_button(self):
        return html.Div(
            id="",
            className="flex justify-between items-center my-2 z-[2]",
            children=[
                html.H2(
                    className="",
                    children=" ",
                ),
                html.Button(
                    id="reset-essential-dis",
                    className="btn btn-sm btn-error",
                    children="Reset",
                ),
            ],
        )

    def _render_buttons(self):
        children = []
        for button in self.model.get_list_of_essential_occ():
            button_id = button
            children.append(
                html.Button(
                    id=button_id,
                    className="btn btn btn-outline btn-sm text-xs",
                    children=button,
                )
            )

        return html.Div(
            className="grid grid-cols-1 p-3 border p-4 border-gray-100 rounded-xl",
            children=children,
        )

    def _render_distribution_chart(self):
        return html.Div(
            className="border p-4 border-gray-100 rounded-xl bg-base-100 col-span-2",
            children=[
                html.H3(
                    id="esstential-pie-title",
                    className="text-xl font-medium text-gray-700",
                    children="title",
                ),
                dcc.Graph(id="essentials-chart", figure={}),
            ],
        )

    def _register_callbacks(self):

        @self.app.callback(
            [
                Output("essentials-chart", "figure"),
                Output("esstential-pie-title", "children"),
            ],
            Input("reset-essential-dis", "n_clicks"),
        )
        def handle_reset_click(n_clicks):
            df = self.model.get_essentails_df_whole()
            fig = create_polar_essentails(df)
            title = "Distribution of Essential Service Human Resources Across Administrative Units"
            return fig, title

        @self.app.callback(
            [
                Output("essentials-chart", "figure", allow_duplicate=True),
                Output("esstential-pie-title", "children", allow_duplicate=True),
            ],
            [Input(f"{i}", "n_clicks") for i in self.model.get_list_of_essential_occ()],
            prevent_initial_call=True,
        )
        def hangle_click(*args):
            if ctx.triggered_id:
                df = self.model.get_essential_by_profession(ctx.triggered_id)
                fig = create_essentials_pie(df)

                title = (
                    f"Distribution of {ctx.triggered_id} Across Administrative Units"
                )
            return fig, title

## View 3

In [ ]:
class GenderDistribution:
    def __init__(self, model, app):
        self.model = model
        self.app = app
        self.layout = self._create_layout()
        self._register_callbacks()
        print(id(self.model))

    def _create_layout(self):
        return html.Div(
            className="container mx-auto card shadow-md hover:shadow-xl rounded-2xl p-6  transition duration-300 border border-gray-100 my-20",
            children=[
                self._render_field_selector(),
                html.Div(
                    className="grid grid-cols-1 lg:grid-cols-3 gap-5",
                    children=[self._render_bar_chart(), self._render_pie_chart()],
                ),
            ],
        )

    def _render_bar_chart(self):
        return html.Div(
            className="grid gap-5 col-span-2 border p-4 border-gray-100 rounded-xl bg-base-100",
            children=[
                html.H2(
                    id="gender-bar-chart-header",
                    className="text-xl font-medium text-gray-700",
                    children="dddd",
                ),
                dcc.Graph(id="graph-gender-distribution", className=""),
                self._render_province_selector(),
            ],
        )

    def _render_pie_chart(self):
        return html.Div(
            className="pt-10 border p-4 border-gray-100 rounded-xl bg-base-100",
            children=[
                dcc.Graph(id="graph-gender-pie"),
                html.H3(
                    id="gender-pie-chart-header",
                    className="label-text font-medium text-gray-700 text-center",
                    children="dsf",
                ),
            ],
        )

    def _render_field_selector(self):
        feilds = self.model.get_field_list()
        return html.Div(
            className="lg:flex justify-between items-center mb-10 z-[2]",
            children=[
                html.H2(
                    className="text-xl font-bold mb-2  text-primary",
                    children="Gender Distribution by Field",
                ),
                dcc.Dropdown(
                    id="field-selector",
                    options=feilds,
                    value=feilds[randint(0, len(feilds) - 1)],
                    clearable=False,
                    className="w-[500px]",
                ),
            ],
        )

    def _render_province_selector(self):
        province_list = self.model.get_province_list()
        return html.Div(
            children=[
                dcc.Dropdown(
                    id="province-selector",
                    value=province_list[:10],
                    options=province_list,
                    multi=True,
                    clearable=False,
                ),
            ],
        )

    def _register_callbacks(self):

        @self.app.callback(
            [
                Output("graph-gender-distribution", "figure"),
                Output("graph-gender-pie", "figure"),
                Output("gender-bar-chart-header", "children"),
                Output("gender-pie-chart-header", "children"),
            ],
            [
                Input("field-selector", "value"),
                Input("province-selector", "value"),
            ],
        )
        def update_figure(field, province_list):
            df_by_province = self.model.get_gender_distribution_by_field(
                field, province_list
            )
            df_canada = self.model.get_gender_distribution_arsc(field)

            fig1 = create_gender_distribution_plot(df_by_province, field)
            fig2 = create_gender_distribution_pie(df_canada, field)

            title_bar = f"Gender Composition of  {field} Across Units"
            tiltle_pie = f"Gender Composition of  {field} Across Canada"
            return fig1, fig2, title_bar, tiltle_pie

## View 4

In [ ]:
class ManpowerDistribution:
    def __init__(self, model, app):
        self.model = model
        self.app = app
        self.layout = self._create_layout()
        print(id(self.model))

    def _create_layout(self):
        df = self.model.get_engineering_df()
        fig = create_tree_map(df)
        return html.Div(
            className="container mx-auto card shadow-md hover:shadow-xl rounded-2xl p-6  transition duration-300 border border-gray-100",
            children=[
                html.H2(
                    className="text-xl font-semibold text-primary mb-5",
                    children="Manpower Analysis for EV Factory Location: Comparing Administrative Units",
                ),
                html.Div(
                    className="grid",
                    children=[
                        dcc.Graph(id="manpower-chart", figure=fig),
                    ],
                ),
            ],
        )

# Main

In [ ]:
# CONTANTS
DATASOURCE_URL = "./data/finalData.csv"


app = dash.Dash(
    __name__,
    external_scripts=JS_SCRIPTS,
    external_stylesheets=SYTLE_SHEETS,
)


model = Model(DATASOURCE_URL)
task01 = EssentialSevericeDistribution(model, app)
task02 = GenderDistribution(model, app)
task03 = ManpowerDistribution(model, app)
task04 = GeneralAnalysis(model, app)

app.title = "CP321 Final Project"
app.layout = [
    html.Div(
        children=[
            html.Div(
                className="pt-20 px-5",
                children=[
                    render_dashboard_header(),
                    task04.layout,
                    task02.layout,
                    task01.layout,
                    task03.layout,
                ],
            ),
            html.Div(id="footer"),
        ]
    )
]

application = app.server
app.run(debug=False)